In [2]:
# -*- coding:utf-8 -*-
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import cv2
import numpy as np
import time

import tensorflow as tf

test_image_dir = './images/'
#model_path = "./model/quantize_frozen_graph.tflite"
model_path = "./output_tflite_graph_helmet.tflite"

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

#with tf.Session( ) as sess:
if 1:
    file_list = os.listdir(test_image_dir)
    
    model_interpreter_time = 0
    img_num = 0
    start_time = time.time()
    # 遍历文件
    for file in file_list:
        print('=========================')
        full_path = os.path.join(test_image_dir, file)
        print('full_path:{}'.format(full_path))
        
        # 只要黑白的，大小控制在(28,28)
        img_scr = cv2.imread(full_path)
        # img = cv2.cvtColor(img_scr,cv2.COLOR_BGR2RGB)
        res_img = cv2.resize(img_scr,(300,300)) 
        # 变成长784的一维数据
        new_img = res_img.reshape(300,300,3)
        
        # 增加一个维度，变为 [1, 784]
        image_np_expanded = np.expand_dims(new_img, axis=0)
        image_np_expanded = image_np_expanded.astype('uint8') # 类型也要满足要求
        
        # 填装数据
        model_interpreter_start_time = time.time()
        interpreter.set_tensor(input_details[0]['index'], image_np_expanded)
        
        # 注意注意，我要调用模型了
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        output_data_1 = interpreter.get_tensor(output_details[1]['index'])
        print('result_1:{}'.format(output_data_1))
        output_data_2 = interpreter.get_tensor(output_details[2]['index'])
        print('result_2:{}'.format(output_data_2))
        output_data_3 = interpreter.get_tensor(output_details[3]['index'])
        print('result_3:{}'.format(output_data_3))
        model_interpreter_time += time.time() - model_interpreter_start_time
        
        # 出来的结果去掉没用的维度
        result = np.squeeze(output_data)
        print('result:{}'.format(result))
        for idx in  range(0,10):
            if output_data_1[0][idx] == 0:
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)
            if output_data_2[0][idx] > 0.5 :
                cv2.rectangle(res_img, (int(output_data[0][idx][1] * 300), int(output_data[0][idx][0] * 300)), (int(output_data[0][idx][3] * 300), int(output_data[0][idx][2] * 300)), color, 2)
                cv2.putText(res_img, "%s: %.2f" % (("helmet" if output_data_2[0][idx] > 0.5 else "without_hemelt" ), output_data_2[0][idx]), 
                        (int(output_data[0][idx][0] * 300) + 6, int(output_data[0][idx][1] * 300)  + 24),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color)
        #print('result:{}'.format(sess.run(output, feed_dict={newInput_X: image_np_expanded})))
        
        # 输出结果是长度为10（对应0-9）的一维数据，最大值的下标就是预测的数字
        print('result:{}'.format( (np.where(result==np.max(result)))[0][0]  ))
        store_path = "./result/result_" + str(img_num) + ".jpeg"
        cv2.imwrite(store_path,res_img)
        img_num += 1
    used_time = time.time() - start_time
    print('used_time:{}'.format(used_time))
    print('model_interpreter_time:{}'.format(model_interpreter_time))


[{'name': 'normalized_input_image_tensor', 'index': 260, 'shape': array([  1, 300, 300,   3], dtype=int32), 'shape_signature': array([  1, 300, 300,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0078125, 128), 'quantization_parameters': {'scales': array([0.0078125], dtype=float32), 'zero_points': array([128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'TFLite_Detection_PostProcess', 'index': 252, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'TFLite_Detection_PostProcess:1', 'index': 253, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parame